In [1]:
# code adapted from https://github.com/ageron/handson-ml/blob/master/10_introduction_to_artificial_neural_networks.ipynb

In [2]:
# number of neurons in the input, hidden, and output layers
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [3]:
import tensorflow as tf

# 'placeholder' nodes to represent the training data and targets
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [4]:
import numpy as np

# function to create a neuron layer
def neuron_layer(X, n_neurons, name, activation=None):
    # name scoping for better visualization in TensorBoard if desired
    with tf.name_scope(name):
        # get the number of inputs
        n_inputs = int(X.get_shape()[1])
        
        # create the weights matrix W and initialize weights randomly
        stddev = 2 / np.sqrt(n_inputs)
        init   = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W      = tf.Variable(init, name="kernel")
        
        # instantiate and initalize biases to 0
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        
        # subgraph for efficient computation of X.W+b
        Z = tf.matmul(X, W) + b
        
        # run activation function (e.g., ReLU) if parameter set
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [5]:
# create a DNN with two hidden layers and a softmax output layer
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

# using a pre-implemented library function instead...
'''
from tensorflow.contrib.layers import fully_connected

with tf.name_scope("dnn"):
    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")
    logits = fully_connected(hidden2, n_outputs, scope="outputs, activation_fn=None")
'''

'\nfrom tensorflow.contrib.layers import fully_connected\n\nwith tf.name_scope("dnn"):\n    hidden1 = fully_connected(X, n_hidden1, scope="hidden1")\n    hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2")\n    logits = fully_connected(hidden2, n_outputs, scope="outputs, activation_fn=None")\n'

In [6]:
# define cross entropy loss function
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [7]:
# instantiate gradient descent loss optimizer
with tf.name_scope("train"):
    learning_rate = 0.01
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [8]:
# specify accuracy as evaluation metric
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [9]:
# load MNIST dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [10]:
# intialize variables
init = tf.global_variables_initializer()

# set number of epochs and batches
n_epochs = 200
batch_size = 50

# train the DNN
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                            y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val)

    # evaluate DNN performance
    X_new_scaled = mnist.test.images[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

    print("Predicted classes:\t", y_pred)
    print("Actual classes:\t\t", mnist.test.labels[:20])

0 Train accuracy: 0.94 Val accuracy: 0.9136
1 Train accuracy: 0.96 Val accuracy: 0.9308
2 Train accuracy: 0.96 Val accuracy: 0.9396
3 Train accuracy: 0.96 Val accuracy: 0.947
4 Train accuracy: 0.98 Val accuracy: 0.9512
5 Train accuracy: 1.0 Val accuracy: 0.9556
6 Train accuracy: 0.94 Val accuracy: 0.9604
7 Train accuracy: 0.98 Val accuracy: 0.963
8 Train accuracy: 0.98 Val accuracy: 0.9636
9 Train accuracy: 0.9 Val accuracy: 0.9664
10 Train accuracy: 1.0 Val accuracy: 0.9702
11 Train accuracy: 0.98 Val accuracy: 0.9706
12 Train accuracy: 0.98 Val accuracy: 0.9722
13 Train accuracy: 0.96 Val accuracy: 0.9718
14 Train accuracy: 1.0 Val accuracy: 0.9742
15 Train accuracy: 1.0 Val accuracy: 0.975
16 Train accuracy: 0.96 Val accuracy: 0.9746
17 Train accuracy: 1.0 Val accuracy: 0.975
18 Train accuracy: 0.96 Val accuracy: 0.976
19 Train accuracy: 0.98 Val accuracy: 0.976
20 Train accuracy: 0.96 Val accuracy: 0.9758
21 Train accuracy: 0.98 Val accuracy: 0.9782
22 Train accuracy: 1.0 Val accur

185 Train accuracy: 1.0 Val accuracy: 0.9818
186 Train accuracy: 1.0 Val accuracy: 0.982
187 Train accuracy: 1.0 Val accuracy: 0.982
188 Train accuracy: 1.0 Val accuracy: 0.9822
189 Train accuracy: 1.0 Val accuracy: 0.9824
190 Train accuracy: 1.0 Val accuracy: 0.982
191 Train accuracy: 1.0 Val accuracy: 0.982
192 Train accuracy: 1.0 Val accuracy: 0.982
193 Train accuracy: 1.0 Val accuracy: 0.982
194 Train accuracy: 1.0 Val accuracy: 0.9822
195 Train accuracy: 1.0 Val accuracy: 0.982
196 Train accuracy: 1.0 Val accuracy: 0.982
197 Train accuracy: 1.0 Val accuracy: 0.982
198 Train accuracy: 1.0 Val accuracy: 0.9822
199 Train accuracy: 1.0 Val accuracy: 0.9822
Predicted classes:	 [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:		 [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
